In [311]:
import pandas as pd
import numpy as np

# Load all files
df_pop = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Population_2023.csv")
df_income = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/median_income.csv")
df_insurance = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Insurance_Coverage_Total_Population_by_state.csv")
df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")
df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")
df_travel = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/state_travel_time.csv")




/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/230149653.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")
/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/230149653.py:9: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")


Working with Population dataset
* Keeping the 50 states and not including purto rico
* need to strip data from state column to remove trailing spaces

In [312]:
df_pop = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Population_2023.csv")

In [313]:
# state column has trailing space so renamed it and rest is clean
# state is a string and population is int so it is good
df_pop.columns = df_pop.columns.str.lower()
df_pop.rename(columns={'state ':'state'},inplace=True)

In [314]:
df_pop['state'] = df_pop['state'].str.strip()

Working with median income dataset
* Converted the column names to lower case
* The income is object -> converted to string
* Income is divided by 1000 to make the numbers small
* changing column name of location -> state



In [315]:
df_income = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/median_income.csv")

In [316]:
df_income.rename(columns={'Location':'location','Median Annual Household Income':'median_income($1k)'},inplace=True)


In [317]:
# median income is string, need to convert it to int and also remove the $
df_income['median_income($1k)'] = (
    df_income['median_income($1k)']
    .replace('[\$,]', '', regex=True)
    .astype(float)
    / 1000
)


In [318]:
df_income = df_income[(df_income['location'] != 'United States') & (df_income['location'] != 'Puerto Rico')]

In [319]:
df_income.rename(columns={'location':'state'},inplace=True)

In [320]:
df_income['state'] = df_income['state'].str.strip()

Working with insurance dataset:
* removing "united states" and purto rico rows
* creating new column by using ~uninsured to get the % of insured people
* keeping only relevant columns : Location and Insured

In [321]:
df_insurance.columns = df_insurance.columns.str.lower()
df_insurance

,location,employer,non-group,medicaid,medicare,military,uninsured,total
0,United States,0.486,0.062,0.212,0.147,0.013,0.079,1
1,Alabama,0.463,0.062,0.207,0.163,0.021,0.084,1
2,Alaska,0.438,0.054,0.230,0.115,0.059,0.105,1
3,Arizona,0.466,0.053,0.204,0.165,0.014,0.098,1
4,Arkansas,0.424,0.057,0.253,0.160,0.015,0.092,1
5,California,0.470,0.066,0.272,0.120,0.008,0.064,1
6,Colorado,0.522,0.068,0.187,0.136,0.020,0.067,1
7,Connecticut,0.513,0.050,0.227,0.148,0.006,0.056,1
8,Delaware,0.479,0.042,0.216,0.187,0.008,0.068,1
9,District of Columbia,0.589,0.043,0.252,0.080,0.008,0.027,1


In [322]:
df_insurance['insured'] = (1 - df_insurance['uninsured']) * 100
df_insurance = df_insurance[['location','insured']]

In [323]:
df_insurance = df_insurance[(df_insurance['location'] != 'United States')& (df_insurance['location'] != 'Puerto Rico')]

In [324]:
df_insurance.rename(columns={'location':'state'},inplace=True)

In [325]:
df_insurance['state'] = df_insurance['state'].str.strip()

Working on Broadband dataset:
* need just two columns: state and brodband%
* cleaning the "geography_desc" column using regex to extract state
* droppping rows where state could not be extracted
* feature engineering by grouping all the states and then finding average broadband%
* mapping the State Abbervation which was extracted to the state name 

In [326]:
df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")
df_broadband

/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/2507160966.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_broadband = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/broadband_by_state.csv")


,geography_type,geography_id,geography_desc,data_type,provider_id,res_st_pct,bus_iv_pct
0,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130228,0.5514,0.5514
1,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130403,0.5697,0.6291
2,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130405,0.0045,0.0045
3,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130627,0.9998,0.9998
4,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130673,0.8001,0.7891
...,...,...,...,...,...,...,...
498899,Tribal,9975R,Wassamasaw SDTSA,Fixed Broadband,430076,1.0000,1.0000
498900,Tribal,9975R,Wassamasaw SDTSA,Mobile Broadband,130077,1.0000,0.8559
498901,Tribal,9975R,Wassamasaw SDTSA,Mobile Broadband,130403,0.9972,0.3814
498902,Tribal,9975R,Wassamasaw SDTSA,Mobile Broadband,131425,1.0000,0.7655


In [327]:
df_broadband['state_abbr'] = df_broadband['geography_desc'].str.extract(r',\s*([A-Z]{2})')
df_broadband

,geography_type,geography_id,geography_desc,data_type,provider_id,res_st_pct,bus_iv_pct,state_abbr
0,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130228,0.5514,0.5514,SD
1,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130403,0.5697,0.6291,SD
2,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130405,0.0045,0.0045,SD
3,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130627,0.9998,0.9998,SD
4,CBSA (MSA),10100,"Aberdeen, SD Micro Area",Fixed Broadband,130673,0.8001,0.7891,SD
...,...,...,...,...,...,...,...,...
498899,Tribal,9975R,Wassamasaw SDTSA,Fixed Broadband,430076,1.0000,1.0000,NaN
498900,Tribal,9975R,Wassamasaw SDTSA,Mobile Broadband,130077,1.0000,0.8559,NaN
498901,Tribal,9975R,Wassamasaw SDTSA,Mobile Broadband,130403,0.9972,0.3814,NaN
498902,Tribal,9975R,Wassamasaw SDTSA,Mobile Broadband,131425,1.0000,0.7655,NaN


In [328]:
df_broadband = df_broadband.dropna(subset=['state_abbr'])

In [329]:
df_broadband = df_broadband.groupby('state_abbr')['res_st_pct'].mean().reset_index().rename(columns={'state_abbr':'state','res_st_pct':'broadband'})

In [330]:
df_broadband = df_broadband[(df_broadband['state']!='PR')&(df_broadband['state']!='GU')&(df_broadband['state']!='MP')&(df_broadband['state']!='AS')&(df_broadband['state']!='VI')]

In [331]:
df_broadband.reset_index(drop=True)

,state,broadband
0,AK,0.753460
1,AL,0.653822
2,AR,0.673949
3,AZ,0.690787
4,CA,0.619892
5,CO,0.655108
6,CT,0.736873
7,DC,0.267852
8,DE,0.817147
9,FL,0.672296


In [332]:
df_broadband['broadband'] = (df_broadband['broadband']*100).round(2)

In [333]:
us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut','DC':'District of Columbia' ,'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

df_broadband['state'] = df_broadband['state'].map(us_state_abbrev)

In [334]:
df_broadband['state'] = df_broadband['state'].str.strip()

In [335]:
df_broadband

,state,broadband
0,Alaska,75.35
1,Alabama,65.38
2,Arkansas,67.39
4,Arizona,69.08
5,California,61.99
6,Colorado,65.51
7,Connecticut,73.69
8,District of Columbia,26.79
9,Delaware,81.71
10,Florida,67.23


Working with Facilities dataset:
* filter and keep only column = Site State Abbreviation
* aggregate that column and count the number of occurances and put it in new column

In [336]:
df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")


/var/folders/vk/fzlwy9hj4ts96ppvlkx7lpmc0000gn/T/ipykernel_98360/894847931.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_facilities = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/Health_Center_Service_Delivery_and_LookAlike_Sites.csv")


In [337]:
df_facilities = df_facilities[['Site State Abbreviation']]

In [338]:
#dropping nan values
df_facilities = df_facilities.dropna(subset=['Site State Abbreviation'])
df_facilities.rename(columns={'Site State Abbreviation':'state'},inplace=True)

In [339]:

df_facilities['state'] = df_facilities['state'].map(us_state_abbrev)

In [340]:
df_facilities = df_facilities.groupby('state').size().reset_index()

In [341]:
df_facilities.rename(columns={0:'facility_count'},inplace=True)

In [342]:
df_facilities['state'] = df_facilities['state'].str.strip()

Working with state travel time dataset:
* converting column names to lower case
* associating the state abbrevations to full name 

In [343]:
df_travel = pd.read_csv("/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/raw/state_travel_time.csv")


In [344]:
df_travel.columns = df_travel.columns.str.lower()

In [345]:
df_travel['state'] = df_travel['state'].map(us_state_abbrev)

In [346]:
df_travel['state'] = df_travel['state'].str.strip()

Merging the clean dataset

In [347]:
df_final = df_pop.copy()

In [348]:
df_pop.columns

Index(['state', 'population_2023'], dtype='object')

In [349]:
# Start with population
df_final = df_pop.copy()

# Merge income
df_final = df_final.merge(df_income, on='state', how='left')

# Merge insurance
df_final = df_final.merge(df_insurance, on='state', how='left')

# Merge broadband (state in lowercase)
df_final = df_final.merge(df_broadband, on='state', how='left')

# Merge health facility count
df_final = df_final.merge(df_facilities, on='state', how='left')

df_final = df_final.merge(df_travel, on='state', how='left')

# Final check



In [350]:
df_final.to_csv('/Users/manika/Desktop/Python : Learning/healthcare-accessibility-index/data/clean/healthcare_accessibility_index.csv', index=False)

In [351]:
df_pop['state']

0               California
1                    Texas
2                  Florida
3                 New York
4             Pennsylvania
5                 Illinois
6                     Ohio
7                  Georgia
8           North Carolina
9                 Michigan
10              New Jersey
11                Virginia
12              Washington
13                 Arizona
14           Massachusetts
15               Tennessee
16                 Indiana
17                Missouri
18                Maryland
19               Wisconsin
20                Colorado
21               Minnesota
22          South Carolina
23                 Alabama
24               Louisiana
25                Kentucky
26                  Oregon
27                Oklahoma
28             Connecticut
29                    Iowa
30                    Utah
31                  Nevada
32                Arkansas
33             Mississippi
34                  Kansas
35              New Mexico
36                Nebraska
3

In [352]:
df_broadband[df_broadband['state']=='District of Columbia']

,state,broadband
8,District of Columbia,26.79


In [354]:
df_final


,state,population_2023,median_income($1k),insured,broadband,facility_count,avg_travel_time_min
0,California,39576757,95.521,93.6,61.99,2867,6.65
1,Texas,29994331,75.780,83.7,66.43,821,5.84
2,Florida,22377419,73.311,89.3,67.23,808,6.92
3,New York,19530351,82.095,95.2,75.42,882,5.52
4,Pennsylvania,12804123,73.824,94.6,73.86,479,7.79
5,Illinois,12585336,80.306,93.9,69.35,534,7.59
6,Ohio,11780017,67.769,93.9,66.83,599,8.45
7,Georgia,10942800,74.632,88.6,66.86,444,5.46
8,North Carolina,10701022,70.804,90.8,68.50,744,6.81
9,Michigan,10050811,69.183,95.6,61.70,478,6.79
